In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0661],
        [-0.0584]], grad_fn=<AddmmBackward0>)

# 参数访问

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3429,  0.0298,  0.1223, -0.1036,  0.0598,  0.1389,  0.2475,  0.1451]])), ('bias', tensor([-0.0701]))])


## 目标参数

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0701], requires_grad=True)
tensor([-0.0701])


In [5]:
net[2].weight.grad == None

True

## ⼀次性访问所有参数

In [6]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


另⼀种访问⽹络参数的⽅式

In [7]:
net.state_dict()['2.bias'].data

tensor([-0.0701])

## 从嵌套块收集参数

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这⾥嵌套
        net.add_module(f'block {i}', block1())
    return net

In [9]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4957],
        [-0.4957]], grad_fn=<AddmmBackward0>)

In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


访问第⼀个主要的块中、第⼆个⼦块的第⼀层的偏置项

In [11]:
rgnet[0][1][0].bias.data

tensor([-0.4466,  0.3225,  0.1833,  0.4948,  0.2619,  0.3483, -0.0172, -0.2974])

# 参数初始化

## 内置初始化

In [12]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

In [13]:
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0110, -0.0281,  0.0148,  0.0129]), tensor(0.))

以将所有参数初始化为给定的常数，⽐如初始化为1

In [14]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

In [15]:
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应⽤不同的初始化⽅法

In [16]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [17]:
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.4651, -0.0588,  0.1503,  0.0853])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 自定义初始化

w ~     U(5,10)     1/4
        0           1/2
        U(-10, -5)  1/4

In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) 
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

In [19]:
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.0778,  0.0000,  0.0000,  7.2936],
        [ 0.0000,  6.7798,  9.0807,  7.2419]], grad_fn=<SliceBackward0>)

可以直接设置参数

In [20]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  8.2936])

## 参数初始化

定义⼀个稠密层，然后使⽤它的参数来设置另⼀个层的参数。

In [21]:
# 我们需要给共享层⼀个名称，以便可以引⽤它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))

In [22]:
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同⼀个对象，⽽不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
